In [58]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [59]:
data = []
for dir, x, files in os.walk("flower_photos"):
    label = dir.split("/")[-1]
    for file in files:
        path = os.path.join(dir, file)
        data.append([path, label])

df = pd.DataFrame(data, columns = ["file_path", "label"])
print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [60]:
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [61]:
train_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)
vaild_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)


Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.


In [62]:
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

In [63]:
early_stopping = EarlyStopping(
    patience = 10,
    min_delta = 0,
    monitor = "val_loss",
    verbose = 0,
    restore_best_weights = True,
    baseline = None
)

In [56]:
model.fit(
    train_gen,
    validation_data = vaild_gen,
    epochs = 5,
    validation_steps = None,
    shuffle = False,
    callbacks = early_stopping,
    verbose = 1
)
#train AI

/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 345s 4s/step - acc: 0.4156 - auc_1: 0.7130 - loss: 1.6639 - precision_1: 0.4789 - recall_1: 0.3164 - val_acc: 0.0954 - val_auc_1: 0.3032 - val_loss: 3.9230 - val_precision_1: 0.0986 - val_recall_1: 0.0763
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 326s 4s/step - acc: 0.7438 - auc_1: 0.9338 - loss: 0.7260 - precision_1: 0.7927 - recall_1: 0.6727 - val_acc: 0.1076 - val_auc_1: 0.3636 - val_loss: 4.3641 - val_precision_1: 0.1141 - val_recall_1: 0.0995
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - acc: 0.8247 - auc_1: 0.9677 - loss: 0.4982 - precision_1: 0.8758 - recall_1: 0.7882 - val_acc: 0.1172 - val_auc_1: 0.3709 - val_loss: 4.8551 - val_precision_1: 0.1239 - val_recall_1: 0.1131
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - acc: 0.8614 - auc_1: 0.9795 - loss: 0.3920 - precision_1: 0.9017 - recall_1: 0.8399 - val_acc: 0.1172 - val_auc_1: 0.3739 - val_loss: 5.1942 - val_precision_1: 0.1232 - val_recall_1: 0.1144
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━

In [57]:
model.save("flowers.keras")
#save AI

In [70]:
import cv2
from tensorflow.keras.saving import load_model
import numpy as np

model = load_model("flowers.keras")

input_image = cv2.imread("/Users/Cohort23/Desktop/Programs/Unit-4-ai/flower_photos/daisy/100080576_f52e8ee070_n.jpg")

input_image_resize = cv2.resize(input_image, (224, 224))

input_image_scaled = input_image_resize/225
image_reshaped = np.reshape(input_image_scaled, [1,224,224,3])

input_prediction = model.predict(image_reshaped)
print("Your flower is a:", np.argmax(input_prediction))

#make the AI make a prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Your flower is a: 1
